In [10]:
import os
import time
import cv2
import mediapipe as mp
import warnings

warnings.filterwarnings("ignore")

# Optional: Use YouTube video or webcam
YOUTUBE_URL = "https://youtu.be/XgLlPN7oxwY?si=887Wd2LF6dax_Kp5" # or None
CROP_DIR = "face_crops"
ANNOTATED_DIR = "annotated_frames" # Directory for full original images

os.makedirs(CROP_DIR, exist_ok=True)
os.makedirs(ANNOTATED_DIR, exist_ok=True) # Create new directory

SAVE_EVERY_N_FRAMES = 50
CROP_PADDING = 25

# ------------------ Try YouTube (yt_dlp optional) ------------------
def get_video_source(url):
    """Return a playable video URL or webcam index (0)."""
    if not url:
        print("🎥 Using webcam.")
        return 0
    try:
        import yt_dlp
        # Use progressive format '22' (720p MP4) or '18' (360p MP4) for better cv2.VideoCapture compatibility
        ydl_opts = {
            'format': '22/18',
            'quiet': True,
            'extractor_args': {'youtube': 'player_client=default'}
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            video_url = info.get('url')
            
            if video_url:
                print("YouTube direct stream found.")
                return str(video_url)
            else:
                raise Exception("No direct stream URL found for selected formats.")
    except Exception as e:
        print(f"YouTube extraction failed: {e}")
        print("Falling back to webcam.")
        return 0

source = get_video_source(YOUTUBE_URL)
cap = cv2.VideoCapture(source)

# Set a small buffer size to improve stability for network streams
if str(source).startswith('http'):
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 3) 

if not cap.isOpened():
    print("Could not open YouTube stream, using webcam instead.")
    cap = cv2.VideoCapture(0)

# ------------------ Mediapipe Setup ------------------
mp_face_mesh = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=10,  # Supports up to 10 faces
    refine_landmarks=True,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

COLORS = [
    (0, 255, 0), (0, 128, 255), (255, 0, 0), (255, 255, 0),
    (203, 192, 255), (145, 255, 0), (45, 110, 255),
    (255, 0, 255), (255, 45, 0), (234, 134, 245)
]

def landmarks_to_bbox(landmarks, w, h, pad=25):
    xs = [lm.x for lm in landmarks]; ys = [lm.y for lm in landmarks]
    x1, x2 = max(0,int(min(xs)*w)-pad), min(w-1,int(max(xs)*w)+pad)
    y1, y2 = max(0,int(min(ys)*h)-pad), min(h-1,int(max(ys)*h)+pad)
    return x1, y1, x2, y2

# ------------------ Main Loop ------------------
print("Stream started — Press 'q' to quit.")
frame_idx, prev_time = 0, 0
save_counters = {}
annotated_saved_this_frame = False # Flag to ensure frame is only saved once per frame_idx

while True:
    ret, frame = cap.read()
    if not ret:
        print("Stream ended or no frames received.")
        break

    # IMPORTANT CHANGE: Create a clean copy before any drawing takes place
    clean_frame = frame.copy() 

    frame_idx += 1
    h, w = frame.shape[:2]
    # Process the 'frame'
    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    annotated_saved_this_frame = False # Reset flag

    if results.multi_face_landmarks:
        for i, face in enumerate(results.multi_face_landmarks):
            color = COLORS[i % len(COLORS)]
            spec = mp_draw.DrawingSpec(color=color, thickness=1, circle_radius=1)

            # Draw annotations (applied to 'frame')
            mp_draw.draw_landmarks(frame, face, mp_face_mesh.FACEMESH_TESSELATION, spec, spec)
            mp_draw.draw_landmarks(frame, face, mp_face_mesh.FACEMESH_CONTOURS, spec, spec)

            # Get and draw bounding box (applied to 'frame')
            x1, y1, x2, y2 = landmarks_to_bbox(face.landmark, w, h, CROP_PADDING)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 1)

            # save face crop and annotated frame occasionally
            if frame_idx - save_counters.get(i, 0) >= SAVE_EVERY_N_FRAMES:
                # 1. Save the individual face crop (from the annotated 'frame')
                crop = frame[y1:y2, x1:x2]
                crop_fname = f"{CROP_DIR}/face{i+1}_f{frame_idx}.jpg"
                cv2.imwrite(crop_fname, crop)
                print(f"Saved face crop: {crop_fname}")

                # 2. Save the full ORIGINAL frame (once per frame_idx)
                if not annotated_saved_this_frame:
                    annotated_fname = f"{ANNOTATED_DIR}/frame_f{frame_idx}.jpg"
                    # Use clean_frame here
                    cv2.imwrite(annotated_fname, clean_frame)
                    print(f"Saved original frame: {annotated_fname}")
                    annotated_saved_this_frame = True # Set flag

                save_counters[i] = frame_idx

    # FPS display
    now = time.time()
    
    # FIX: Use explicit check to prevent ZeroDivisionError on the first frame
    if prev_time > 0:
        time_diff = now - prev_time
        if time_diff > 1e-6: 
            fps = 1.0 / time_diff
        else:
            fps = 0.0
    else:
        fps = 0.0
        
    prev_time = now
    
    # Draw FPS onto the annotated 'frame' for display
    cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("Face Mesh Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
face_mesh.close()
print("Done! Crops saved in:", os.path.abspath(CROP_DIR))
print("Original frames saved in:", os.path.abspath(ANNOTATED_DIR))

YouTube direct stream found.
Stream started — Press 'q' to quit.
Saved face crop: face_crops/face1_f50.jpg
Saved original frame: annotated_frames/frame_f50.jpg
Saved face crop: face_crops/face2_f65.jpg
Saved original frame: annotated_frames/frame_f65.jpg
Saved face crop: face_crops/face1_f100.jpg
Saved original frame: annotated_frames/frame_f100.jpg
Saved face crop: face_crops/face2_f116.jpg
Saved original frame: annotated_frames/frame_f116.jpg
Saved face crop: face_crops/face1_f150.jpg
Saved original frame: annotated_frames/frame_f150.jpg
Saved face crop: face_crops/face2_f166.jpg
Saved original frame: annotated_frames/frame_f166.jpg
Saved face crop: face_crops/face1_f200.jpg
Saved original frame: annotated_frames/frame_f200.jpg
Saved face crop: face_crops/face1_f271.jpg
Saved original frame: annotated_frames/frame_f271.jpg
Saved face crop: face_crops/face1_f321.jpg
Saved original frame: annotated_frames/frame_f321.jpg
Saved face crop: face_crops/face1_f630.jpg
Saved original frame: a


## ⚙️ Code Status and Functionality Review

Your current script is fully functional and addresses all previous issues and feature requests:

1.  **Multi-Face Processing (10 max)**: Achieved via `max_num_faces=10` in `mp_face_mesh.FaceMesh`.
2.  **Dual Output Saving**:
      * **Crops** are saved to `face_crops`.
      * **Original Frames** (clean image without mesh) are saved to `annotated_frames` using `clean_frame = frame.copy()`.
3.  **1.0x Speed (Every Frame)**: Achieved by setting `FRAME_SKIP_COUNT = 1` and `FRAME_PROCESS_COUNT = 1`.
4.  **Resizable Window**: Implemented using `cv2.namedWindow("Face Mesh Stream", cv2.WINDOW_NORMAL)`.
5.  **Save Frequency Control**: The line `# SAVE_EVERY_N_FRAMES = 50` controls that saving to disk occurs only once every 50 frames, reducing I/O load.

In [11]:
import os
import time
import cv2
import mediapipe as mp
import warnings
warnings.filterwarnings("ignore")

# Optional: Use YouTube video or webcam
YOUTUBE_URL = "https://youtu.be/RdUPs9e1bUk?si=bpDrtuv27A3Jq-Rr" # or None
CROP_DIR = "face_crops"
ANNOTATED_DIR = "annotated_frames" # Directory for full original images

os.makedirs(CROP_DIR, exist_ok=True)
os.makedirs(ANNOTATED_DIR, exist_ok=True) # Create new directory

SAVE_EVERY_N_FRAMES = 50  # Specifies that a face crop and its corresponding original frame will be saved only once every 50 frames.
CROP_PADDING = 25

# ------------------ Speed Control ------------------
# To simulate 1.5x speed, we process 2 out of every 3 frames (skipping 1/3 of frames).
#FRAME_SKIP_COUNT = 3
#FRAME_PROCESS_COUNT = 2

# ------------------ Speed Control ------------------
# To simulate 1x speed, we process every frame (1 out of every 1 frame).
FRAME_SKIP_COUNT = 1
FRAME_PROCESS_COUNT = 1

# ------------------ Try YouTube (yt_dlp optional) ------------------
def get_video_source(url):
    """Return a playable video URL or webcam index (0)."""
    if not url:
        print("Using webcam.")
        return 0
    try:
        import yt_dlp
        # Use progressive format '22' (720p MP4) or '18' (360p MP4) for better cv2.VideoCapture compatibility
        ydl_opts = {
            'format': '22/18',
            'quiet': True,
            'extractor_args': {'youtube': 'player_client=default'}
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            video_url = info.get('url')
            
            if video_url:
                print("YouTube direct stream found.")
                return str(video_url)
            else:
                raise Exception("No direct stream URL found for selected formats.")
    except Exception as e:
        print(f"YouTube extraction failed: {e}")
        print("Falling back to webcam.")
        return 0

source = get_video_source(YOUTUBE_URL)
cap = cv2.VideoCapture(source)

# Set a small buffer size to improve stability for network streams
if str(source).startswith('http'):
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 3) 

if not cap.isOpened():
    print("Could not open YouTube stream, using webcam instead.")
    cap = cv2.VideoCapture(0)

# ------------------ Mediapipe Setup ------------------
mp_face_mesh = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=10,  # Supports up to 10 faces
    refine_landmarks=True,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

COLORS = [
    (0, 255, 0), (0, 128, 255), (255, 0, 0), (255, 255, 0),(203, 192, 255),
    (145, 255, 0), (45, 110, 255), (255, 0, 255), (255, 45, 0), (234, 134, 245)
]

def landmarks_to_bbox(landmarks, w, h, pad=25):
    xs = [lm.x for lm in landmarks]; ys = [lm.y for lm in landmarks]
    x1, x2 = max(0,int(min(xs)*w)-pad), min(w-1,int(max(xs)*w)+pad)
    y1, y2 = max(0,int(min(ys)*h)-pad), min(h-1,int(max(ys)*h)+pad)
    return x1, y1, x2, y2

# ------------------ Main Loop ------------------
print("Stream started — Press 'q' to quit.")
print("Playback is set to approximately 1.5x speed via frame skipping.")

frame_idx, prev_time = 0, 0
save_counters = {}
annotated_saved_this_frame = False

# Create a resizable window
cv2.namedWindow("Face Mesh Stream", cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Stream ended or no frames received.")
        break

    frame_idx += 1
    
    # 1.5x Speed Implementation: Skip 1 frame for every 3 frames read
    if (frame_idx % FRAME_SKIP_COUNT) > FRAME_PROCESS_COUNT - 1:
        continue # Skip this frame

    # Create a clean copy before any drawing takes place
    clean_frame = frame.copy() 

    h, w = frame.shape[:2]
    # Process the 'frame'
    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    annotated_saved_this_frame = False # Reset flag

    if results.multi_face_landmarks:
        for i, face in enumerate(results.multi_face_landmarks):
            color = COLORS[i % len(COLORS)]
            spec = mp_draw.DrawingSpec(color=color, thickness=1, circle_radius=1)

            # Draw annotations (applied to 'frame')
            mp_draw.draw_landmarks(frame, face, mp_face_mesh.FACEMESH_TESSELATION, spec, spec)
            mp_draw.draw_landmarks(frame, face, mp_face_mesh.FACEMESH_CONTOURS, spec, spec)

            # Get and draw bounding box (applied to 'frame')
            x1, y1, x2, y2 = landmarks_to_bbox(face.landmark, w, h, CROP_PADDING)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 1)

            # save face crop and annotated frame occasionally
            if frame_idx - save_counters.get(i, 0) >= SAVE_EVERY_N_FRAMES:
                # 1. Save the individual face crop (from the annotated 'frame')
                crop = frame[y1:y2, x1:x2]
                crop_fname = f"{CROP_DIR}/face{i+1}_f{frame_idx}.jpg"
                cv2.imwrite(crop_fname, crop)
                print(f"Saved face crop: {crop_fname}")

                # 2. Save the full ORIGINAL frame (once per frame_idx)
                if not annotated_saved_this_frame:
                    annotated_fname = f"{ANNOTATED_DIR}/frame_f{frame_idx}.jpg"
                    # Use clean_frame here (original image without annotations)
                    cv2.imwrite(annotated_fname, clean_frame)
                    print(f"Saved original frame: {annotated_fname}")
                    annotated_saved_this_frame = True # Set flag

                save_counters[i] = frame_idx

    # FPS display
    now = time.time()
    
    # Use explicit check to prevent ZeroDivisionError on the first frame
    if prev_time > 0:
        time_diff = now - prev_time
        if time_diff > 1e-6: 
            # Multiply FPS by the inverse of the frames skipped to get actual processed FPS
            processed_fps = 1.0 / time_diff
            # Note: The true visual speed is 1.5x, but this FPS is the processing rate.
            fps = processed_fps
        else:
            fps = 0.0
    else:
        fps = 0.0
        
    prev_time = now
    
    # Draw FPS onto the annotated 'frame' for display
    cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("Face Mesh Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
face_mesh.close()
print("Done! Crops saved in:", os.path.abspath(CROP_DIR))
print("Original frames saved in:", os.path.abspath(ANNOTATED_DIR))

YouTube direct stream found.
Stream started — Press 'q' to quit.
Playback is set to approximately 1.5x speed via frame skipping.
Saved face crop: face_crops/face1_f220.jpg
Saved original frame: annotated_frames/frame_f220.jpg
Saved face crop: face_crops/face1_f270.jpg
Saved original frame: annotated_frames/frame_f270.jpg
Saved face crop: face_crops/face2_f312.jpg
Saved original frame: annotated_frames/frame_f312.jpg
Saved face crop: face_crops/face1_f320.jpg
Saved original frame: annotated_frames/frame_f320.jpg
Saved face crop: face_crops/face1_f370.jpg
Saved original frame: annotated_frames/frame_f370.jpg
Saved face crop: face_crops/face1_f552.jpg
Saved original frame: annotated_frames/frame_f552.jpg
Saved face crop: face_crops/face1_f602.jpg
Saved original frame: annotated_frames/frame_f602.jpg
Saved face crop: face_crops/face1_f652.jpg
Saved original frame: annotated_frames/frame_f652.jpg
Saved face crop: face_crops/face1_f721.jpg
Saved original frame: annotated_frames/frame_f721.jp